# Fashion RAG - Préparation de la Base Vectorielle

Ce notebook guide la création d'une base de données vectorielle ChromaDB pour un système RAG (Retrieval-Augmented Generation) de recommandation d'articles de mode.

## Objectifs
1. Charger et explorer les données fashion
2. Créer des descriptions textuelles complètes pour chaque article
3. Construire et persister la base vectorielle ChromaDB
4. Tester des requêtes de similarité

## 1. Installation des dépendances

In [ ]:
# Installation des packages nécessaires
!pip install pyarrow==14.0.2 --only-binary=:all:
!pip install numpy==1.24.1


!pip install chromadb langchain langchain-openai langchain-community pandas python-dotenv tiktoken

## 2. Imports et Configuration

In [1]:
import pandas as pd
import os
from dotenv import load_dotenv

# ChromaDB
import chromadb
from chromadb.config import Settings

# LangChain
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_core.documents import Document

# Configuration
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPEN_AI_API_KEY")


/Users/aba/Documents/Cours Ib Cegos/IA Generative - Les modeles de langages massifs LLMs/20251201/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 3. Chargement et Exploration des Données

In [2]:
# Chemins des fichiers
STYLES_PATH = "data/styles.csv"
IMAGES_PATH = "data/images.csv"

# Chargement des données
df_styles = pd.read_csv(STYLES_PATH, on_bad_lines='skip')
df_images = pd.read_csv(IMAGES_PATH)

print(f"Styles dataset: {len(df_styles)} articles")
print(f"Images dataset: {len(df_images)} images")

Styles dataset: 44424 articles
Images dataset: 44446 images


In [3]:
# Aperçu des données styles
print("=" * 50)
print("STYLES DATASET")
print("=" * 50)
print(f"\nColonnes: {df_styles.columns.tolist()}")
print(f"\nTypes de données:\n{df_styles.dtypes}")
df_styles.head()

STYLES DATASET

Colonnes: ['id', 'gender', 'masterCategory', 'subCategory', 'articleType', 'baseColour', 'season', 'year', 'usage', 'productDisplayName']

Types de données:
id                      int64
gender                 object
masterCategory         object
subCategory            object
articleType            object
baseColour             object
season                 object
year                  float64
usage                  object
productDisplayName     object
dtype: object


,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName
0,15970,Men,Apparel,Topwear,Shirts,Navy Blue,Fall,2011.0,Casual,Turtle Check Men Navy Blue Shirt
1,39386,Men,Apparel,Bottomwear,Jeans,Blue,Summer,2012.0,Casual,Peter England Men Party Blue Jeans
2,59263,Women,Accessories,Watches,Watches,Silver,Winter,2016.0,Casual,Titan Women Silver Watch
3,21379,Men,Apparel,Bottomwear,Track Pants,Black,Fall,2011.0,Casual,Manchester United Men Solid Black Track Pants
4,53759,Men,Apparel,Topwear,Tshirts,Grey,Summer,2012.0,Casual,Puma Men Grey T-shirt


In [5]:
# Aperçu des données images
print("=" * 50)
print("IMAGES DATASET")
print("=" * 50)
print(f"\nColonnes: {df_images.columns.tolist()}")
df_images.head()

IMAGES DATASET

Colonnes: ['filename', 'link']


,filename,link
0,15970.jpg,http://assets.myntassets.com/v1/images/style/p...
1,39386.jpg,http://assets.myntassets.com/v1/images/style/p...
2,59263.jpg,http://assets.myntassets.com/v1/images/style/p...
3,21379.jpg,http://assets.myntassets.com/v1/images/style/p...
4,53759.jpg,http://assets.myntassets.com/v1/images/style/p...


In [6]:
# Statistiques sur les catégories
print("\nDistribution par catégorie principale:")
print(df_styles['masterCategory'].value_counts())

print("\nDistribution par genre:")
print(df_styles['gender'].value_counts())

print("\nTop 10 couleurs:")
print(df_styles['baseColour'].value_counts().head(10))


Distribution par catégorie principale:
masterCategory
Apparel           21397
Accessories       11274
Footwear           9219
Personal Care      2403
Free Items          105
Sporting Goods       25
Home                  1
Name: count, dtype: int64

Distribution par genre:
gender
Men       22147
Women     18631
Unisex     2161
Boys        830
Girls       655
Name: count, dtype: int64

Top 10 couleurs:
baseColour
Black        9728
White        5538
Blue         4918
Brown        3494
Grey         2741
Red          2455
Green        2115
Pink         1860
Navy Blue    1789
Purple       1640
Name: count, dtype: int64


## 4. Fusion et Nettoyage des Données

In [4]:
# Extraction de l'ID depuis le filename des images
df_images['id'] = df_images['filename'].str.replace('.jpg', '').astype(int)

# Fusion des deux datasets
df_merged = pd.merge(df_styles, df_images, on='id', how='left')

print(f"Dataset fusionné: {len(df_merged)} articles")
print(f"Articles avec images: {df_merged['link'].notna().sum()}")
print(f"Articles sans images: {df_merged['link'].isna().sum()}")

Dataset fusionné: 44424 articles
Articles avec images: 44424
Articles sans images: 0


In [5]:
# Nettoyage des données
def clean_dataframe(df: pd.DataFrame) -> pd.DataFrame:
    """
    Nettoie le dataframe pour la préparation RAG.
    """
    df_clean = df.copy()
    
    # Supprimer les lignes sans ID ou sans nom de produit
    df_clean = df_clean.dropna(subset=['id', 'productDisplayName'])
    
    # Remplir les valeurs manquantes
    fill_values = {
        'gender': 'Unisex',
        'masterCategory': 'Unknown',
        'subCategory': 'Unknown',
        'articleType': 'Unknown',
        'baseColour': 'Unknown',
        'season': 'All Season',
        'year': 'Unknown',
        'usage': 'Casual',
        'link': ''
    }
    df_clean = df_clean.fillna(fill_values)
    
    # Convertir l'ID en int
    df_clean['id'] = df_clean['id'].astype(int)
    
    # Nettoyer les espaces
    string_columns = ['gender', 'masterCategory', 'subCategory', 'articleType', 
                      'baseColour', 'season', 'usage', 'productDisplayName']
    for col in string_columns:
        if col in df_clean.columns:
            df_clean[col] = df_clean[col].astype(str).str.strip()
    
    return df_clean

df_clean = clean_dataframe(df_merged)
print(f"Dataset nettoyé: {len(df_clean)} articles")

Dataset nettoyé: 44417 articles


In [9]:
# Vérification du nettoyage
print("\nVérification des valeurs manquantes après nettoyage:")
print(df_clean.isnull().sum())


Vérification des valeurs manquantes après nettoyage:
id                    0
gender                0
masterCategory        0
subCategory           0
articleType           0
baseColour            0
season                0
year                  0
usage                 0
productDisplayName    0
filename              0
link                  0
dtype: int64


## 5. Création des Descriptions Textuelles pour le RAG

In [6]:
def create_product_description(row: pd.Series) -> str:
    """
    Crée une description textuelle complète d'un produit pour l'embedding.
    Cette description sera utilisée pour la recherche sémantique.
    """
    # Extraction de la marque (premier mot du productDisplayName)
    product_name = str(row['productDisplayName'])
    brand = product_name.split()[0] if product_name else 'Unknown Brand'
    
    # Construction de la description enrichie
    description_parts = [
        f"Product: {product_name}",
        f"Brand: {brand}",
        f"Type: {row['articleType']}",
        f"Category: {row['masterCategory']} - {row['subCategory']}",
        f"Color: {row['baseColour']}",
        f"Gender: {row['gender']}",
        f"Season: {row['season']}",
        f"Usage: {row['usage']}",
    ]
    
    # Ajouter l'année si disponible
    if row['year'] != 'Unknown' and pd.notna(row['year']):
        description_parts.append(f"Year: {int(row['year']) if isinstance(row['year'], float) else row['year']}")
    
    return ". ".join(description_parts)

# Créer la colonne description
df_clean['description'] = df_clean.apply(create_product_description, axis=1)

# Aperçu des descriptions générées
print("Exemples de descriptions générées:\n")
for i, desc in enumerate(df_clean['description'].head(3)):
    print(f"Article {i+1}:")
    print(f"{desc}\n")
    print("-" * 80)

Exemples de descriptions générées:

Article 1:
Product: Turtle Check Men Navy Blue Shirt. Brand: Turtle. Type: Shirts. Category: Apparel - Topwear. Color: Navy Blue. Gender: Men. Season: Fall. Usage: Casual. Year: 2011

--------------------------------------------------------------------------------
Article 2:
Product: Peter England Men Party Blue Jeans. Brand: Peter. Type: Jeans. Category: Apparel - Bottomwear. Color: Blue. Gender: Men. Season: Summer. Usage: Casual. Year: 2012

--------------------------------------------------------------------------------
Article 3:
Product: Titan Women Silver Watch. Brand: Titan. Type: Watches. Category: Accessories - Watches. Color: Silver. Gender: Women. Season: Winter. Usage: Casual. Year: 2016

--------------------------------------------------------------------------------


## 6. Préparation des Documents LangChain

In [7]:
def create_langchain_documents(df: pd.DataFrame) -> list:
    """
    Convertit le dataframe en liste de Documents LangChain avec métadonnées.
    """
    documents = []
    
    for _, row in df.iterrows():
        # Métadonnées pour le filtrage et l'affichage
        metadata = {
            'id': int(row['id']),
            'product_name': str(row['productDisplayName']),
            'brand': str(row['productDisplayName']).split()[0] if row['productDisplayName'] else 'Unknown',
            'article_type': str(row['articleType']),
            'master_category': str(row['masterCategory']),
            'sub_category': str(row['subCategory']),
            'color': str(row['baseColour']),
            'gender': str(row['gender']),
            'season': str(row['season']),
            'usage': str(row['usage']),
            'year': str(row['year']),
            'image_url': str(row['link']) if pd.notna(row['link']) else '',
            'filename': str(row['filename']) if 'filename' in row and pd.notna(row['filename']) else ''
        }
        
        doc = Document(
            page_content=row['description'],
            metadata=metadata
        )
        documents.append(doc)
    
    return documents

# Création des documents
documents = create_langchain_documents(df_clean)
print(f"{len(documents)} documents LangChain créés")

44417 documents LangChain créés


In [8]:
# Aperçu d'un document
print("Exemple de document LangChain:\n")
sample_doc = documents[0]
print(f"Contenu: {sample_doc.page_content}\n")
print(f"Métadonnées: {sample_doc.metadata}")

Exemple de document LangChain:

Contenu: Product: Turtle Check Men Navy Blue Shirt. Brand: Turtle. Type: Shirts. Category: Apparel - Topwear. Color: Navy Blue. Gender: Men. Season: Fall. Usage: Casual. Year: 2011

Métadonnées: {'id': 15970, 'product_name': 'Turtle Check Men Navy Blue Shirt', 'brand': 'Turtle', 'article_type': 'Shirts', 'master_category': 'Apparel', 'sub_category': 'Topwear', 'color': 'Navy Blue', 'gender': 'Men', 'season': 'Fall', 'usage': 'Casual', 'year': '2011.0', 'image_url': 'http://assets.myntassets.com/v1/images/style/properties/7a5b82d1372a7a5c6de67ae7a314fd91_images.jpg', 'filename': '15970.jpg'}


## 7. Création de la Base Vectorielle ChromaDB

In [13]:
# Configuration de ChromaDB
PERSIST_DIRECTORY = "./chroma_db"
COLLECTION_NAME = "fashion_products"

# Initialisation des embeddings OpenAI
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small",  # Modèle économique et efficace
    # on peut aussi utiliser "text-embedding-3-large" pour plus de précision
)

print("Embeddings OpenAI initialisés")

Embeddings OpenAI initialisés


In [9]:
## Exportation des documents en JSONL pour vérification ou usage ultérieur
import json
with open("data/docs.jsonl", "w", encoding="utf-8") as f:
    for d in documents:
        f.write(json.dumps({"page_content": d.page_content, "metadata": d.metadata}) + "\n")


In [14]:
# Création de la base vectorielle (peut prendre quelques minutes selon la taille du dataset)
# Pour un grand dataset, on peut limiter le nombre de documents pour le test

# Option 1: Utiliser tout le dataset (recommandé pour la production)
# docs_to_index = documents

# Option 2: Limiter pour les tests (plus rapide)
MAX_DOCS = 5000  # A Ajuster selon les besoins et le quota API
docs_to_index = documents[:MAX_DOCS]

print(f"Indexation de {len(docs_to_index)} documents...")

Indexation de 5000 documents...


In [15]:
# Création et persistance de la base vectorielle
import time

start_time = time.time()

# Création de la base vectorielle avec persistance
vectorstore = Chroma.from_documents(
    documents=docs_to_index,
    embedding=embeddings,
    persist_directory=PERSIST_DIRECTORY,
    collection_name=COLLECTION_NAME
)

elapsed_time = time.time() - start_time
print(f"\nBase vectorielle créée et persistée en {elapsed_time:.2f} secondes")
print(f"Emplacement: {PERSIST_DIRECTORY}")


Base vectorielle créée et persistée en 24.87 secondes
Emplacement: ./chroma_db


## 8. Test de Recherche par Similarité

In [16]:
def test_similarity_search(query: str, k: int = 5):
    """
    Teste la recherche par similarité et affiche les résultats.
    """
    print(f"\nRequête: '{query}'")
    print("=" * 80)
    
    # Recherche avec scores
    results = vectorstore.similarity_search_with_score(query, k=k)
    
    for i, (doc, score) in enumerate(results, 1):
        print(f"\nRésultat {i} (Score de similarité: {score:.4f})")
        print(f"   Produit: {doc.metadata['product_name']}")
        print(f"   Type: {doc.metadata['article_type']}")
        print(f"   Couleur: {doc.metadata['color']}")
        print(f"   Genre: {doc.metadata['gender']}")
        print(f"   Saison: {doc.metadata['season']}")
        print(f"   Usage: {doc.metadata['usage']}")
        if doc.metadata['image_url']:
            print(f"   Image: {doc.metadata['image_url']}")
    
    return results

In [17]:
# Test 1: Recherche en anglais
results_1 = test_similarity_search("casual blue jeans for men", k=3)


Requête: 'casual blue jeans for men'

Résultat 1 (Score de similarité: 0.7305)
   Produit: John Players Men Blue Jeans
   Type: Jeans
   Couleur: Blue
   Genre: Men
   Saison: Summer
   Usage: Casual
   Image: http://assets.myntassets.com/v1/images/style/properties/John-Players-Men-Blue-Jeans_48c4929e8a62d1b8ea6070c400c9e06e_images.jpg

Résultat 2 (Score de similarité: 0.7305)
   Produit: John Players Men Blue Jeans
   Type: Jeans
   Couleur: Blue
   Genre: Men
   Saison: Summer
   Usage: Casual
   Image: http://assets.myntassets.com/v1/images/style/properties/John-Players-Men-Blue-Jeans_48c4929e8a62d1b8ea6070c400c9e06e_images.jpg

Résultat 3 (Score de similarité: 0.7381)
   Produit: Jack & Jones Men Blue Jeans
   Type: Jeans
   Couleur: Blue
   Genre: Men
   Saison: Summer
   Usage: Casual
   Image: http://assets.myntassets.com/v1/images/style/properties/1e85047e167a6e042302be3a4a6c8da8_images.jpg


In [18]:
# Test 2: Recherche en français (le modèle d'embedding gère le multilingue)
results_2 = test_similarity_search("robe rouge élégante pour femme", k=3)


Requête: 'robe rouge élégante pour femme'

Résultat 1 (Score de similarité: 0.9995)
   Produit: Red Rose Women Pink & White Bath Robes
   Type: Bath Robe
   Couleur: White
   Genre: Women
   Saison: Summer
   Usage: Casual
   Image: http://assets.myntassets.com/v1/images/style/properties/Red-Rose-Women-Pink-Polka-Dotted-Bath-Robe_f3a716f2580dde1b768c8aabb40a0c64_images.jpg

Résultat 2 (Score de similarité: 0.9996)
   Produit: Red Rose Women Pink & White Bath Robes
   Type: Bath Robe
   Couleur: White
   Genre: Women
   Saison: Summer
   Usage: Casual
   Image: http://assets.myntassets.com/v1/images/style/properties/Red-Rose-Women-Pink-Polka-Dotted-Bath-Robe_f3a716f2580dde1b768c8aabb40a0c64_images.jpg

Résultat 3 (Score de similarité: 1.0035)
   Produit: Red Rose Women White & Pink Polka Dot Print Bath Robe
   Type: Bath Robe
   Couleur: Pink
   Genre: Women
   Saison: Winter
   Usage: Casual
   Image: http://assets.myntassets.com/v1/images/style/properties/Red-Rose-Women-Bath-Robe_b47

In [19]:
# Test 3: Recherche par style/occasion
results_3 = test_similarity_search("formal black shoes for office", k=3)


Requête: 'formal black shoes for office'

Résultat 1 (Score de similarité: 0.8132)
   Produit: Clarks Men Black Formal Shoes
   Type: Formal Shoes
   Couleur: Black
   Genre: Men
   Saison: Fall
   Usage: Formal
   Image: http://assets.myntassets.com/v1/images/style/properties/7fbee63ac47052f962bd5f25aa8652b8_images.jpg

Résultat 2 (Score de similarité: 0.8132)
   Produit: Clarks Men Black Formal Shoes
   Type: Formal Shoes
   Couleur: Black
   Genre: Men
   Saison: Fall
   Usage: Formal
   Image: http://assets.myntassets.com/v1/images/style/properties/7fbee63ac47052f962bd5f25aa8652b8_images.jpg

Résultat 3 (Score de similarité: 0.8270)
   Produit: Arrow Men Formal Black Shoes
   Type: Formal Shoes
   Couleur: Black
   Genre: Men
   Saison: Fall
   Usage: Formal
   Image: http://assets.myntassets.com/v1/images/style/properties/ae9b19f9e5b20995dd206e0400b2dbce_images.jpg


In [20]:
# Test 4: Recherche saisonnière
results_4 = test_similarity_search("summer dress light colors", k=3)


Requête: 'summer dress light colors'

Résultat 1 (Score de similarité: 1.0196)
   Produit: 109F Women Multi Coloured Dress
   Type: Dresses
   Couleur: Multi
   Genre: Women
   Saison: Summer
   Usage: Casual
   Image: http://assets.myntassets.com/v1/images/style/properties/f00a9de0191d5bdf9bd182eaf4298c72_images.jpg

Résultat 2 (Score de similarité: 1.0196)
   Produit: 109F Women Multi Coloured Dress
   Type: Dresses
   Couleur: Multi
   Genre: Women
   Saison: Summer
   Usage: Casual
   Image: http://assets.myntassets.com/v1/images/style/properties/f00a9de0191d5bdf9bd182eaf4298c72_images.jpg

Résultat 3 (Score de similarité: 1.0274)
   Produit: Sepia Women Pink Dress
   Type: Dresses
   Couleur: Pink
   Genre: Women
   Saison: Summer
   Usage: Casual
   Image: http://assets.myntassets.com/v1/images/style/properties/c1a94e1415f58fc183bc9488bf692e5b_images.jpg


In [21]:
# Test 5: Recherche par sport
results_5 = test_similarity_search("je veux des shorts de sport pour l'été. Elles doivent être noir et courtes", k=3)


Requête: 'je veux des shorts de sport pour l'été. Elles doivent être noir et courtes'

Résultat 1 (Score de similarité: 0.9897)
   Produit: Nike Boys Basketball Black Shorts
   Type: Shorts
   Couleur: Black
   Genre: Boys
   Saison: Summer
   Usage: Sports
   Image: http://assets.myntassets.com/v1/images/style/properties/b8b2ceebfb49c01db50d6da07780dd2a_images.jpg

Résultat 2 (Score de similarité: 0.9899)
   Produit: Nike Boys Basketball Black Shorts
   Type: Shorts
   Couleur: Black
   Genre: Boys
   Saison: Summer
   Usage: Sports
   Image: http://assets.myntassets.com/v1/images/style/properties/b8b2ceebfb49c01db50d6da07780dd2a_images.jpg

Résultat 3 (Score de similarité: 1.0031)
   Produit: Nike Men Black Shorts
   Type: Shorts
   Couleur: Black
   Genre: Men
   Saison: Summer
   Usage: Casual
   Image: http://assets.myntassets.com/v1/images/style/properties/42f6d3632908535c4c099a4d68070984_images.jpg


## 9. Affichage des Images (si disponibles)

In [22]:
from IPython.display import Image, display, HTML

def display_results_with_images(results, max_display=3):
    """
    Affiche les résultats avec leurs images.
    """
    html_content = "<div style='display: flex; flex-wrap: wrap; gap: 20px;'>"
    
    for i, (doc, score) in enumerate(results[:max_display], 1):
        image_url = doc.metadata.get('image_url', '')
        product_name = doc.metadata.get('product_name', 'Unknown')
        color = doc.metadata.get('color', 'Unknown')
        article_type = doc.metadata.get('article_type', 'Unknown')
        
        html_content += f"""
        <div style='border: 1px solid #ddd; padding: 10px; border-radius: 8px; width: 250px;'>
            <h4 style='margin: 0 0 10px 0;'>#{i} - Score: {score:.4f}</h4>
            {'<img src="' + image_url + '" style="width: 100%; height: 200px; object-fit: contain;" />' if image_url else '<p style="color: gray;">No image available</p>'}
            <p><strong>{product_name[:40]}...</strong></p>
            <p>Type: {article_type}</p>
            <p>Couleur: {color}</p>
        </div>
        """
    
    html_content += "</div>"
    display(HTML(html_content))

# Affichage des résultats de la recherche "casual blue jeans"
print("Visualisation des résultats pour 'casual blue jeans for men':")
display_results_with_images(results_1)

Visualisation des résultats pour 'casual blue jeans for men':


## 10. Rechargement de la Base Vectorielle (pour l'application)

In [27]:
# Démonstration du rechargement de la base persistée

def load_vectorstore(persist_directory: str, collection_name: str):
    """
    Charge une base vectorielle existante depuis le disque.
    """
    embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
    
    vectorstore = Chroma(
        persist_directory=persist_directory,
        embedding_function=embeddings,
        collection_name=collection_name
    )
    
    return vectorstore

# Test de rechargement
loaded_vectorstore = load_vectorstore(PERSIST_DIRECTORY, COLLECTION_NAME)
print(f"Base vectorielle rechargée avec succès!")

# Vérification avec une requête
test_results = loaded_vectorstore.similarity_search("white sneakers", k=2)
print(f"\nTest après rechargement:")
for doc in test_results:
    print(f"   - {doc.metadata['product_name']}")

Base vectorielle rechargée avec succès!

Test après rechargement:
   - Skechers Men Black Sneakers
   - Skechers Men Black Sneakers


## 11. Statistiques de la Base Vectorielle

In [24]:
# Récupération des statistiques
collection = vectorstore._collection

print("Statistiques de la base vectorielle:")
print(f"   Nom de la collection: {COLLECTION_NAME}")
print(f"   Nombre de documents: {collection.count()}")
print(f"   Répertoire de persistance: {PERSIST_DIRECTORY}")

Statistiques de la base vectorielle:
   Nom de la collection: fashion_products
   Nombre de documents: 10000
   Répertoire de persistance: ./chroma_db
